# Machine Learning Project by Teodor Chakarov

# Fake and real news dataset

Fake news are problem nowadays for this missinforamtion that we have all around the Internet and social media. Canwe really distinguish the difference between real and fake news. Can we make a good model for NLP so we can easily say if we can trust one news' source or not?

So I will explore and clean 2 csv files that i downloaded from "https://www.kaggle.com/clmentbisaillon/fake-and-real-news-dataset". 

In [ ]:
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import seaborn as sns
import datetime 

In [ ]:
import re, string
import nltk 
import nltk as nlp

import unicodedata
from nltk.corpus import stopwords
from collections import Counter
from wordcloud import WordCloud, STOPWORDS , ImageColorGenerator

from sklearn.decomposition import PCA

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV

from sklearn.linear_model import LogisticRegression, SGDClassifier

from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, plot_confusion_matrix, plot_roc_curve, make_scorer, confusion_matrix

from sklearn.pipeline import Pipeline

from sklearn.base import BaseEstimator, is_classifier, is_regressor, is_outlier_detector

In [ ]:
np.random.seed(42)

In [ ]:
fake_news = pd.read_csv('/kaggle/input/fake-and-real-news-dataset/Fake.csv')
real_news = pd.read_csv('/kaggle/input/fake-and-real-news-dataset/True.csv')

In [ ]:
fake_news

In [ ]:
real_news

I will merge those 2 datasets and I will add another column for category (Fake: 0, Real: 1) 

In [ ]:
fake_news['category'] = 0
real_news['category'] = 1

In [ ]:
news = pd.concat([fake_news, real_news])

In [ ]:
news = news.reset_index()

In [ ]:
news = news.drop('index', axis = 1)

In [ ]:
news

In [ ]:
news.isna().sum()

In [ ]:
plt.figure(figsize=(8,6))

plt.title('By category')

sns.countplot(news.category)

plt.ylabel("Count")

plt.show()

The Dataset is well banalced in terms of news category also we don't have NaN values which i good 

#### Analysing the date

I noticed that I can't covert to pd.to_datetime because i have different date column records.

In [ ]:
news[news['date'].str.len() > 19]

Basically I will create new variable with those 10 records droped for date analysis.

In [ ]:
http_filter = news['date'].str.contains('http')

In [ ]:
news_date = news[~http_filter]

In [ ]:
news_date = news_date[~news_date.date.str.contains('MSNB')]

In [ ]:
news_date

In [ ]:
news_date['date'] = pd.to_datetime(news_date.date)

In [ ]:
news_date

#### Distribution of the news in the time period

In [ ]:
sns.displot(news_date, x=news_date.date, hue="category", element = 'step', height=8.27, aspect=11.7/8.27)

We can see the destribution for the news through time and we can see that real news we have much move form period 2019/09 to 2018/01 but for fake we have more frequent int the past 

#### Count of news per year per news category

In [ ]:
plt.close()
plt.figure(figsize=(8,6))
sns.countplot(news_date.date.dt.year, data=news_date, hue='category')
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
sns.set(style = "whitegrid",font_scale = 1.2)

plt.title('Number of news per subject')

sns.countplot(x= 'subject', hue = 'category', data = news)

plt.ylabel("Count")

plt.show()

We can see we have different kind of subjects for both true and fake news. We are going to delete this column in the generel text we are going to inspect.

I'm going to combine text and title columns so we can inspect the whole text in general

In [ ]:
raw_txt = pd.DataFrame()

In [ ]:
raw_txt['category'] = news.category
raw_txt['text'] = news['text'] + " " + news['title']

In [ ]:
raw_txt['text'] = raw_txt.apply(lambda x: x['text'].lower(),axis=1)

In [ ]:
raw_txt.text = raw_txt.text.str.replace('[^\w\s]','')

In [ ]:
raw_txt

## General text analisys

### Data Cleaning

#### Perform data cleaning process so we can proceed with ML part

In [ ]:
stop = set(stopwords.words('english'))

In [ ]:
def stemming(text):
    wnl = nlp.stem.WordNetLemmatizer()
    stopwords = nlp.corpus.stopwords.words('english')
    text = (unicodedata.normalize('NFKD', text)
    .encode('ascii', 'ignore')
    .decode('utf-8', 'ignore')
    .lower())
    words = re.sub(r'[^\w\s]', '', text).split()
    return [wnl.lemmatize(word) for word in words if word not in stopwords]

In [ ]:
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)
#Removing the noisy text
def denoise_text(text):
    text = remove_stopwords(text).lower()
    re.sub(r'http\S+', '', text)
    re.sub(r'[^\w\s]', '', text)
    wnl = nlp.stem.WordNetLemmatizer()
    
    return wnl.lemmatize(text)

In [ ]:
def GetModelScores (estimator, X_train, y_train):
    from sklearn.model_selection import cross_validate
    
    estimator.fit(X_train, y_train)
    
    scoring = {'acc': 'accuracy','f1_score': 'f1_weighted','roc_auc': 'roc_auc'}
    
    scores = cross_validate(estimator, X_train, y_train, cv=5, scoring = scoring,  return_train_score=True)
    
            
    #print(scores.keys())
    print(f"Accuracity on Training: {scores['train_acc'].max()} and Validating: {scores['test_acc'].max()}")
    print(f"F1_Score on Training: {scores['train_f1_score'].max()} and Validating: {scores['test_f1_score'].max()}")
    print(f"ROC_AUC Score on Training: {scores['train_roc_auc'].max()} and Validating: {scores['test_roc_auc'].max()}")

In [ ]:
class ClfSwitcher(BaseEstimator):

    def __init__(self, estimator = LogisticRegression()):

        self.estimator = estimator


    def fit(self, X, y=None, **kwargs):
        self.estimator.fit(X, y)
        return self


    def predict(self, X, y=None):
        return self.estimator.predict(X)


    def predict_proba(self, X):
        return self.estimator.predict_proba(X)


    def score(self, X, y):
        return self.estimator.score(X, y)

In [ ]:
fake_words = stemming(''.join(str(raw_txt[raw_txt.category == 0]['text'].tolist())))
real_words = stemming(''.join(str(raw_txt[raw_txt.category == 1]['text'].tolist())))

I joined all the rows for each fake and real news to list.

In [ ]:
raw_txt['text']=raw_txt['text'].apply(denoise_text)

Performed general denoisation in the text

#### N_Grams for Real News and Fake

Biagram Fake news

In [ ]:
pd.Series(nlp.ngrams(fake_words,2)).value_counts()[:30]

Biagram Real news

In [ ]:
pd.Series(nlp.ngrams(real_words,2)).value_counts()[:30]

Triagram Fake news

In [ ]:
pd.Series(nlp.ngrams(fake_words,3)).value_counts()[:30]

Triagram Real news

In [ ]:
pd.Series(nlp.ngrams(real_words,3)).value_counts()[:30]

Here we saw how frequent 2 or 3 words we can see in the text. Most of them are in USA and aiming presidents and overseas relations. 

#### Cloud of Words

In [ ]:
# Start with one review:
generic_words = " ".join(raw_txt[raw_txt.category == 1].text)

plt.figure(figsize  = (15,10))
# Create and generate a word cloud image:
wordcloud_ALL = WordCloud(max_font_size=500, max_words=500, background_color="black", width = 1600 , height = 800).generate(
    generic_words)
plt.imshow(wordcloud_ALL, interpolation='bilinear')
plt.title('Popular REAL news words', fontsize=30)
plt.axis('off')
plt.show()

In [ ]:
# Start with one review:
generic_words = " ".join(raw_txt[raw_txt.category == 0].text)

plt.figure(figsize  = (15,10))
# Create and generate a word cloud image:
wordcloud_ALL = WordCloud(max_font_size=500, max_words=500, background_color="black", width = 1600 , height = 800).generate(
    generic_words)
plt.imshow(wordcloud_ALL, interpolation='bilinear')
plt.title('Popular FAKE news words', fontsize=30)
plt.axis('off')
plt.show()

In [ ]:
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(12,8))
text_len=raw_txt[raw_txt['category']==1]['text'].str.split().map(lambda x: len(x))
ax1.hist(text_len,color='red')
ax1.set_title('Real text')
text_len=raw_txt[raw_txt['category']==0]['text'].str.split().map(lambda x: len(x))
ax2.hist(text_len,color='green')
ax2.set_title('Fake text')
fig.suptitle('Words in texts')
plt.show()

See words frequency in the text. Fake news are greather than Real news.

## Machine Learning

In this part we are going to see which algorithm is the best and is there a significant difference between TFIDF and Count Vectorisers.

In [ ]:
X = raw_txt['text']
y = raw_txt['category']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, shuffle = True,
                                                   stratify = y, test_size = 0.15)

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
X_train_vectorised = TfidfVectorizer().fit_transform(X_train)

In [ ]:
X_train

In [ ]:
print(X_train_vectorised)

The tuple represents: (document_id, token_id)

The value following the tuple represents the tf-idf score of a given token in a given document

The tuples that are not there have a tf-idf score of 0

In [ ]:
GetModelScores(LogisticRegression(), X_train_vectorised, y_train)

In [ ]:
GetModelScores(DecisionTreeClassifier(), X_train_vectorised, y_train)

In [ ]:
GetModelScores(MultinomialNB(), X_train_vectorised, y_train)

In [ ]:
GetModelScores(SGDClassifier(), X_train_vectorised, y_train)

I have really good scores without any tuning?! Let's see with a little bit am I going to overfitt a lot or ... ?

In [ ]:
pipeline = Pipeline([
    
    ('vect', TfidfVectorizer()),
    ('clf', ClfSwitcher())
    
])

### 1) Logistic Rregression

In [ ]:
parameters_log = [

    {
        'clf__estimator': [LogisticRegression()],
        'vect__max_df': [0.75],
        'clf__estimator__penalty': ['l2'],
        'clf__estimator__C': [1,10,100]
        
    }
]

In [ ]:
gscv_log = GridSearchCV(pipeline, parameters_log, scoring = 'f1')
gscv_log.fit(X_train, y_train)

In [ ]:
gscv_log.best_estimator_

In [ ]:
gscv_log.best_score_

### 2) Decision Trees

In [ ]:
parameters_DT = [
    {
        'clf__estimator': [DecisionTreeClassifier()], 
        'vect__max_df': [0.75],
        'clf__estimator__criterion': ['entropy'],
        'clf__estimator__max_depth': [2, 20, 50, 100],
        'clf__estimator__min_samples_leaf': [1, 2, 4]  
    }
]

In [ ]:
gscv_DT = GridSearchCV(pipeline, parameters_DT,  scoring = 'f1')

In [ ]:
gscv_DT.fit(X_train, y_train)

In [ ]:
gscv_DT.best_estimator_

In [ ]:
gscv_DT.best_score_

## MODEL EVALUATION 

In [ ]:
def ModelElavuation(estimator, X_train, y_train, X_test, y_test):
    from sklearn.metrics import plot_confusion_matrix
    
    
    scores_test = pd.DataFrame(columns= ['Accuracy','F1 Score','Precision','Recall', 'ROC_AUC'])
    
    try:
        score_train = estimator.predict_proba(X_train)[:,1]
        roc_train= roc_auc_score(y_train, score_train, average = "weighted")
    except:
        roc_train = 0
        
    try:
        score_test = model.predict_proba(X_test)[:,1]
        roc_test= roc_auc_score(y_test, score_test, average = "weighted")
    except:
        roc_test = 0

    scores_test['Accuracy'] = accuracy_score(y_test, estimator.predict(X_test))*100,
    scores_test['F1 Score'] = f1_score(y_test, estimator.predict(X_test), average = "weighted")*100,
    scores_test['Precision'] = precision_score(y_test, estimator.predict(X_test), average = "weighted")*100,
    scores_test['Recall'] = recall_score(y_test, estimator.predict(X_test), average = "weighted")*100,
    scores_test['ROC_AUC'] = roc_train*100
    

    print(scores_test)
    print("")
    plt.title('Train set')
    sns.heatmap(confusion_matrix(y_train, estimator.predict(X_train)), annot = True)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()
    print("")
    plt.title('Test set')
    sns.heatmap(confusion_matrix(y_test, estimator.predict(X_test)), annot = True)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

In [ ]:
ModelElavuation(gscv_log, X_train, y_train, X_test, y_test)

In [ ]:
ModelElavuation(gscv_DT, X_train, y_train, X_test, y_test)

## Conclusion (1)

What I heave GREAT results with those 2 ML algorithms???? But are they really great once or mabye Fake news and Real news have key distinguishable differences? How is it possible to predict that great those 2 categories?

## Further Inspection

I trained only the text column since the date if I input True news before 2016 I can be missclassified or the news' subject as well. Also the subject column is not accurate at all as we saw we have only 2 for Real News

Let me just go back to Real news dataset and Fake one :

In [ ]:
real_news

In [ ]:
fake_news

We can see with naked eye the (Reuters) in the real news dataset. Reuters is internation information and news provider organization. Maybe it's some kind of CODE word for real news. Let's see the % of this wor in both sets:

In [ ]:
print('Real news have: ', real_news.text.str.contains('(Reuters)').sum()/len(real_news))
print('Fake news have: ', fake_news.text.str.contains('(Reuters)').sum()/len(fake_news))

Yes so basically it really show the test set what to category should be and maybe other observations are those who are misclassified.

In the fake news set I see 21st Century Wire... Let's see the % again.

In [ ]:
print('Real news have: ', real_news.text.str.contains('21st Century').sum()/len(real_news))
print('Fake news have: ', fake_news.text.str.contains('21st Century').sum()/len(fake_news))

Let's inspect for dublicated text and remove those rows.

In [ ]:
news[news.duplicated()]

We have to get rid of these duplicated rows because mabye they can go to training and testing sets. We will have leaked label to the validation/training set. The number of rows are 209, it should be risky but let's see the results.

In [ ]:
plt.title('Dublicated observations for category')
sns.countplot(news[news.duplicated()]['category'])
plt.show()

The majority is in the real news. Before droping the duplicated rows we have advantage for Fake news and after dropping them we will have 206 less and it's going to unbalance more. 

In [ ]:
news_adjusted = news.drop_duplicates()

In [ ]:
news_adjusted

#### I will remove (Reuters) and (21st Century Wire) in each oservation so I can make it more fair and less high biased data.

In [ ]:
news_adjusted['text'] = news_adjusted['title'] + ' ' + news_adjusted['text'] 

In [ ]:
news_adjusted['text'] = news_adjusted['text'].replace('(Reuters)', '')

In [ ]:
news_adjusted['text'] = news_adjusted['text'].replace('21st Century Wire', '')

In [ ]:
news_adjusted.drop('title', axis  = 1)

## Machine Learning (2)

Let's try again to see if we can gat less high biased models

In [ ]:
news_adjusted['text']=news_adjusted['text'].apply(denoise_text)

In [ ]:
X_adjusted = news_adjusted['text']
y_adjusted = news_adjusted['category']

In [ ]:
X_adjusted_train, X_adjusted_test, y_adjusted_train, y_adjusted_test = train_test_split(X_adjusted, y_adjusted,
                                random_state = 42, shuffle = True, stratify = y_adjusted, test_size = 0.15)

In [ ]:
X_adjusted_train.shape, y_adjusted_train.shape, X_adjusted_test.shape, y_adjusted_test.shape

In [ ]:
X_adjusted_train_vectorised = TfidfVectorizer().fit_transform(X_adjusted_train)

In [ ]:
GetModelScores(LogisticRegression(), X_adjusted_train_vectorised, y_adjusted_train)

In [ ]:
GetModelScores(DecisionTreeClassifier(), X_adjusted_train_vectorised, y_adjusted_train)

In [ ]:
parameters_DT = [
    {
        'clf__estimator': [DecisionTreeClassifier()], 
        'vect__max_df': [0.75],
        'clf__estimator__criterion': ['entropy'],
        'clf__estimator__max_depth': [2, 20, 50, 100],
        'clf__estimator__min_samples_leaf': [1, 2, 4]  
    }
]

In [ ]:
gscv_DT_adjusted = GridSearchCV(pipeline, parameters_DT,  scoring = 'f1')

In [ ]:
gscv_DT_adjusted.fit(X_adjusted_train, y_adjusted_train)

In [ ]:
gscv_DT_adjusted.best_score_

In [ ]:
ModelElavuation(gscv_DT_adjusted, X_adjusted_train, y_adjusted_train, X_adjusted_test, y_adjusted_test)

## Conclusion|

So basically in general we have perfect models for only this dataset. I inspected some of the data leakage. The method for gathering data is not good because as we saw, we have "KEY" words (like Reuters or 21st Century) which indicates the model. We don't want this high bias or "cheat tuned" data. We saw with naked eye but obviusly the problem is still there, even though we did some more cleaning. I would suggest for more adequate data gathering and not putting "CHEATING" words in there. 

In news filter we have to really be careful for misclassification because for False Positive will lead to backlash from the author/publisher and their readers who trust them, resulting in distrust of the filter and their administrator.
Or for False Negatives Real news will not get to the people and they will miss maybe important information.  